In [345]:
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from numpy.random import rand,randint
import scipy.ndimage
from scipy.sparse import csc_matrix, csr_matrix
import json
import matplotlib.pyplot as plt
import queue
import functools
import pandas
import math

In [ ]:
RANDOM_SEED = 42
cluster_number = 256
weights = rand(300,784)

### Clustering

In [ ]:
def reshape_weights_for_kmeans(weights):
    return np.hstack(weights).reshape(-1,1)

def build_clusters(cluster,weights):
    kmeans = MiniBatchKMeans(n_clusters=cluster,random_state=RANDOM_SEED)
    kmeans.fit(reshape_weights_for_kmeans(weights))
    return kmeans.cluster_centers_

In [ ]:
def nearest_centroid_index(centers,value):
    centers = np.asarray(centers)
    return (np.abs(centers - value)).argmin()

def nearest_centroid(centers,value):
    centers = np.asarray(centers)
    idx = (np.abs(centers - value)).argmin()
    return centers[idx]

### Matrix

In [ ]:
def redefine_weights(weights,centers):
    arr_ret = np.empty_like(weights).astype(np.int16)
    for i, row in enumerate(weights):
        for j, col in enumerate(row):
            arr_ret[i,j] = nearest_centroid_index(centers,weights[i,j])
    return arr_ret

In [ ]:
def idx_matrix_to_matrix(idx_matrix,centers,shape):
    return centers[idx_matrix.reshape(-1,1)].reshape(shape)

In [ ]:
def centroid_gradient_matrix(idx_matrix,gradient,cluster):
    return scipy.ndimage.sum(gradient,idx_matrix,index=range(cluster))

### Dictionary

In [ ]:
# map (i,j) -> k
def dict_index_to_cluster(weights,centers):
        dict_ret = {}
        for i, row in enumerate(weights):
            for j, col in enumerate(row):
                dict_ret[(i,j)] = nearest_centroid_index(centers,weights[i,j])
        return dict_ret

# map k -> (i,j)
def dict_cluster_to_index(dict_idx):
    dict_ret = {}
    for k,v in dict_idx.items():
        if v in dict_ret:
            dict_ret[v] += [k]
        else:
            dict_ret[v] = [k]
    return dict_ret

In [ ]:
def index_dict_to_matrix(dict_index,dict_values,shape):
    coord_array = np.asarray(list(dict_index.values()))
    return dict_values[coord_array].reshape(shape)

In [ ]:
def centroid_gradient_dict(K_Index,W_Matrix):
    tmpindex = dict((key, ([x for x, _ in value], [y for _, y in value])) for key, value in K_Index.items())
    return [W_Matrix[value[0],value[1]].sum() for value in tmpindex.values()]

### Variables

In [ ]:
centers = build_clusters(cluster_number,weights)
dict_index = dict_index_to_cluster(weights,centers)
dict_cluster = dict_cluster_to_index(dict_index)
matrix_index = redefine_weights(weights,centers)

### Testing

In [ ]:
%timeit idx_matrix_to_matrix(matrix_index,centers,(300,784))
%timeit index_dict_to_matrix(dict_index,centers,(300,784))

In [ ]:
%timeit centroid_gradient_matrix(matrix_index,weights,cluster_number)
%timeit centroid_gradient_dict(dict_cluster,weights)

### Transform

In [ ]:
def dict_to_index_matrix_slow(dict_cluster):
    arr_ret = np.zeros((300,785))
    for k,v in dict_cluster.items():
        for i in v:
            arr_ret[i] = k
    return arr_ret

def dict_to_index_matrix(dict_index,shape):
    return np.asarray(list(dict_index.values())).reshape(shape).astype(np.int16)

In [ ]:
%timeit dict_to_index_matrix_slow(dict_cluster)
%timeit dict_to_index_matrix(dict_index,(300,784))

# HELPER FUNCTION

#### Cluster mean

In [ ]:
def mean_distance(weights,centroids):
    tot = 0.
    for i, row in enumerate(weights):
        for j, col in enumerate(row):
            weight = weights[i,j]
            centroid = nearest_centroid(centroids,weight)[0]
            tot += np.sqrt((weight - centroid)**2)
    return tot / ((i+1)*(j+1))

In [ ]:
def find_clusters_number(values,n_from,n_to,n_jump):
    result = {}
    for i in range(n_from,n_to+1,n_jump):
        kmeans = MiniBatchKMeans(n_clusters=i,random_state=RANDOM_SEED)
        kmeans.fit(reshape_weights_for_kmeans(values))
        mean = mean_distance(values,kmeans.cluster_centers_)
        result[i] = mean
        print("Mean for %s clusters %f " % (str(i).zfill(3),mean))
    return result
        
means_cluster_1_10 = find_clusters_number(weights,1,101,10)

## Huffman Coding

In [ ]:
cluster_sparsity = [(len(dict_cluster[x]),x) for x in dict_cluster]
cluster_sparsity = sorted(cluster_sparsity, key=lambda x: x[0] )
plt.bar([x[1] for x in cluster_sparsity], [x[0] for x in cluster_sparsity], width=1)
plt.show()

In [ ]:
plt.bar(range(len(cluster_sparsity)), [x[0] for x in sorted(cluster_sparsity, key=lambda x: x[0], reverse=True)], width=1)
plt.show()

In [ ]:
@functools.total_ordering
class HuffmanNode(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right
    def children(self):
        return((self.left, self.right))
    def __lt__(self, other):
        return True

def create_tree(frequencies):
    p = queue.PriorityQueue()
    for value in frequencies:
        p.put(value)
    while p.qsize() > 1:
        l, r = p.get(), p.get()
        node = HuffmanNode(l, r)
        p.put((l[0]+r[0], node))    
    return p.get()

# Dictionary (n : "01010")
def coding_tree(node, prefix="", code={}):
    if isinstance(node[1].left[1], HuffmanNode):
        coding_tree(node[1].left,prefix+"0", code)
    else:
        code[node[1].left[1]]=prefix+"0"
    if isinstance(node[1].right[1],HuffmanNode):
        coding_tree(node[1].right,prefix+"1", code)
    else:
        code[node[1].right[1]]=prefix+"1"
    return(code)

def decode(rev_huff,code):
    for k, v in rev_huff.items():
        if v == code:
            return k
        
def encode(rev_huff,code):
    return rev_huff[code]

# Dictionary ("01010" : n)
def reverse_code(huff):
    huff_code_rev = {}    
    for k,v in huff.items():
        huff_code_rev[v] = k
    return huff_code_rev

def rev_encode(rev_huff,code):
    return decode(rev_huff,code)
        
def rev_decode(rev_huff,code):
    return encode(rev_huff,code)

In [ ]:
cluster_sparsity = [(len(v),k) for k,v in dict_cluster.items()]
ht_cluster = create_tree(cluster_sparsity)
hc_cluster = coding_tree(ht_cluster,code={})
hc_rev = reverse_code(hc_cluster)

list(hc_cluster.items())[0:10]

In [ ]:
def nearest_centroid_index_huffman(centers,value):
    centers = np.asarray(centers)
    idx = (np.abs(centers - value)).argmin()
    return encode(hc_cluster,idx)

def redefine_weights_huffman(weights,centers):
    arr_ret = np.empty_like(weights).astype(str)
    for i, row in enumerate(weights):
        for j, col in enumerate(row):
            arr_ret[i,j] = nearest_centroid_index_huffman(centers,weights[i,j])
    return arr_ret

def dict_index_to_cluster_huffman(weights,centers):
        dict_ret = {}
        for i, row in enumerate(weights):
            for j, col in enumerate(row):
                dict_ret[(i,j)] = nearest_centroid_index_huffman(centers,weights[i,j])
        return dict_ret
    
def index_dict_to_matrix_huffman(dict_index,dict_values,huff_code,shape):
    coord_array = np.asarray([rev_decode(huff_code,x) for x in huff_dict_index.values()])
    return dict_values[coord_array].reshape(shape)

def centroid_gradient_matrix_huffman(idx_huff_matrix,gradient,cluster):
    return scipy.ndimage.sum(gradient,idx_huff_matrix,[x for x in cluster.values()])

huff_index_matrix = redefine_weights_huffman(weights,centers)
huff_dict_index = dict_index_to_cluster_huffman(weights,centers)

In [ ]:
%timeit index_dict_to_matrix(dict_index,centers,(300,784))
%timeit index_dict_to_matrix_huffman(huff_dict_index,centers,hc_rev,(300,784))

In [ ]:
%timeit centroid_gradient_matrix(matrix_index,weights,cluster_number)
%timeit centroid_gradient_matrix_huffman(huff_index_matrix,weights,hc_cluster)

In [ ]:
# Per Huffman non è possibile fare nulla (implementazione)
# Alla fine di tutto calcolare un'ipotetica compressione (funzione che restituisce il tasso)
# sommatoria: grandezza di ogni cluster (# elementi) * lunghezza stringa in char (8b a char)
# in confronto con # elementi * 8b

## PRUNING (with kmeans support)

In [ ]:
def pruning_matrix(mat,percentage,method='out'):
    threshold = (100-percentage)
    
    if method == 'inout':
        threshold /= 4
        perc_up,perc_down,perc_mid_up,perc_mid_down = 100 - threshold, threshold, 50 + threshold, 50 - threshold
        percentile_up = np.percentile(mat,perc_up)
        percentile_down = np.percentile(mat,perc_down)
        percentile_mid_up = np.percentile(mat,perc_mid_up)
        percentile_mid_down = np.percentile(mat,perc_mid_down)
    else:
        threshold /= 2
        if method == 'in': perc_up, perc_down = 50 + threshold, 50 - threshold
        elif method == 'out': perc_up, perc_down = 100 - threshold, threshold
        percentile_up = np.percentile(mat,perc_up)
        percentile_down = np.percentile(mat,perc_down)
        
    w_pruned = np.copy(mat)
    for i,row in enumerate(mat):
        for j,_ in enumerate(row):
            if method == 'in':
                if mat[i,j] > percentile_down and mat[i,j] < percentile_up:
                    w_pruned[i,j] = 0
            elif method == 'out':
                if mat[i,j] < percentile_down or mat[i,j] > percentile_up:
                    w_pruned[i,j] = 0
            elif method == 'inout':
                if mat[i,j] < percentile_down or mat[i,j] > percentile_up or (mat[i,j] > percentile_mid_down and mat[i,j] < percentile_mid_up):
                    w_pruned[i,j] = 0
    return w_pruned

In [ ]:
w_csc = csc_matrix(pruning_matrix(weights,70,method='in'))

In [ ]:
def P_nearest_centroid_index(centers,value):
    centers = np.asarray(centers)
    return (np.abs(centers - value)).argmin()

In [ ]:
def P_build_clusters(cluster,weights):
    kmeans = MiniBatchKMeans(n_clusters=cluster,random_state=RANDOM_SEED)
    kmeans.fit(weights.data.reshape(-1,1))
    return kmeans.cluster_centers_

P_centers = P_build_clusters(256,w_csc)

In [ ]:
def P_redefine_weights(weights,centers):
    new_data_idx = [nearest_centroid_index(centers,w) for w in weights.data]
    return csc_matrix((new_data_idx,weights.indices,weights.indptr))

idx_csc = P_redefine_weights(w_csc,P_centers)

In [327]:
def P_idx_matrix_to_matrix(idx_matrix,centers):
    return csc_matrix((centers[idx_matrix.data].reshape(-1,),idx_matrix.indices,idx_matrix.indptr))

P_idx_matrix_to_matrix(idx_csc,P_centers)

<300x784 sparse matrix of type '<class 'numpy.float64'>'
	with 164640 stored elements in Compressed Sparse Column format>

In [328]:
def P_centroid_gradient_matrix(idx_matrix,gradient,mask,cluster):
    gradient[mask] = 0
    return scipy.ndimage.sum(csc_matrix(gradient).data,idx_matrix.data,index=range(cluster))

mask = w_csc.A == 0
gradient = weights.copy()
cgm = P_centroid_gradient_matrix(idx_csc,gradient,mask,256)

In [ ]:
idx_csc

## CSC Matrix

In [ ]:
# multiplication
# csr * dense

In [ ]:
a = np.array(range(21)).reshape(3,7)
b = np.array(range(20,41)).reshape(3,7)
acc = csc_matrix(a)
acr = csr_matrix(a)
acc.A

In [278]:
# in place
def sparse_sub_dense(sparse,dense,mask):
    sparse.A -= dense
    sparse.A[mask] = 0

def sparse_sub_dense2(sparse,dense,mask):
    b = sparse.A - dense
    b[mask] = 0
    return csc_matrix(b)

# in place
def sparse_sub_dense3(sparse,dense,mask):
    sparse.data -= dense.T.reshape(1,-1)[mask.T.reshape(1,-1)]
    
# in place
def sparse_sub_dense4(sparse,dense,mask):
    sparse.data -= dense.T[mask.T]

In [279]:
t = csc_matrix([[1,2,0],[3,0,1]])
mt = t.A != 0 
rt = np.array([[2,3,4],[5,6,7]])

s1 = sparse_sub_dense(t,rt,mt)
print(t)
print('_________________________________')

t = csc_matrix([[1,2,0],[3,0,1]])
s2 = sparse_sub_dense2(t,rt,mt)
print(s2)
print('_________________________________')

t = csc_matrix([[1,2,0],[3,0,1]])
s3 = sparse_sub_dense3(t,rt,mt)
print(t)
print('_________________________________')

t = csc_matrix([[1,2,0],[3,0,1]])
s4 = sparse_sub_dense4(t,rt,mt)
print(t)

  (0, 0)	1
  (1, 0)	3
  (0, 1)	2
  (1, 2)	1
_________________________________
  (1, 1)	-6
  (0, 2)	-4
_________________________________
  (0, 0)	-1
  (1, 0)	-2
  (0, 1)	-1
  (1, 2)	-6
_________________________________
  (0, 0)	-1
  (1, 0)	-2
  (0, 1)	-1
  (1, 2)	-6


In [280]:
cc = w_csc.copy()
maskBig = cc.A != 0 
r = rand(300,784)
%timeit sparse_sub_dense(cc,r,maskBig)
%timeit sparse_sub_dense2(cc,r,maskBig)
%timeit sparse_sub_dense3(cc,r,maskBig)
%timeit sparse_sub_dense4(cc,r,maskBig)

1.73 ms ± 3.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.62 ms ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.57 ms ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.16 ms ± 3.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
cc - rand(300,784)[mask]

In [ ]:
def delete_last_row_csc(matrix):
    i = matrix.indptr[-1]
    indptr = matrix.indptr[:-1]
    data = matrix.data[:i]
    indices = matrix.indices[:i]
    return csc_matrix((data,indices,indptr))

In [ ]:
v = np.array([1,2,1,1,2,1,1])
np.inner(acc.A,v),acc*v.T,acc*v

In [ ]:
m = [[1,2,0],[3,0,1]]
mc = csc_matrix(m)
ma = mc.A == 0
mc.A -= [[1,1,1],[1,0,0]]
mc.A[ma] = 0
mc.A

In [ ]:
m2 = [[1,2,0],[3,0,1]]
mc2 = csc_matrix(m2)
sparse_subtract_dense(mc2,[[1,1,1],[1,0,0]],ma)
m2,mc2.data

In [ ]:
a = np.array([[1,2,3],[4,5,6]])
a.T.reshape(1,-1)

In [276]:
a = np.array([[1,2,3],[4,5,6]]).T
m = np.array([[True,True,False],[True,False,True]]).T

In [277]:
a[m]

array([1, 4, 2, 6])

In [365]:
def cr(p,k):
    return (784*300)*64/((784*300*p)*math.log(k,2) + k*64)

def result(p,k,t,tr,te):
    print(1,"&",p,"&",k,"&",cr(p,k),"&",round(t/16.46,3),"&",round(tr/98.76,3),"&",round(te/97.69,3),"\\\\")

# 60 - 256 - Epoch 004 (33m13s) Accuracy TRAIN: 99.33%	Accuracy TEST: 98.03%	Min: 96.13% (9)
# 60 - 512 - Epoch 003 (25m8s)  Accuracy TRAIN: 99.37%	Accuracy TEST: 97.97%	Min: 95.74% (9)
# 60 - 1024 - Epoch 005 (43m6s)  Accuracy TRAIN: 99.41%	Accuracy TEST: 98.03%	Min: 96.04% (9)

# 75 - 256 - Epoch 006 (58m29s) Accuracy TRAIN: 99.3%	Accuracy TEST: 97.99%	Min: 96.43% (9)
# 75 - 512 - Epoch 004 (46m33s) Accuracy TRAIN: 99.34%	Accuracy TEST: 97.89%	Min: 96.23% (9)
# 75 - 1024 - Epoch 005 (60m21s) Accuracy TRAIN: 99.36%	Accuracy TEST: 97.98%	Min: 96.43% (9)

# 90 - 256 - Epoch 005 (55m36s) Accuracy TRAIN: 84.34%	Accuracy TEST: 84.42%	Min: 63.45% (5)
# 90 - 512 - Epoch 004 (43m30s) Accuracy TRAIN: 99.33%	Accuracy TEST: 97.96%	Min: 96.53% (9)
# 90 - 1024 - Epoch 004 (43m38s) Accuracy TRAIN: 99.35%	Accuracy TEST: 98.01%	Min: 95.94% (9)

result(0.90,1024,43.38,99.35,98.01)

1 & 0.9 & 1024 & 6.897562978386463 & 2.635 & 1.006 & 1.003 \\


In [350]:
(784*300)*64/((784*300)*math.log(512,2) + 512*64),(784*300)*64/((784*300*0.5)*math.log(512,2) + 512*64)

(7.002709381605978, 13.795125956772926)

In [348]:
math.log(256,2)

8.0

In [355]:
(784*300)/(784*300*0.45)

2.2222222222222223